In [1]:
import numpy as np
import pandas as pd
from utils import *

np.set_printoptions(precision=3, suppress=True)
pd.set_option('display.float_format', '{:.3f}'.format)

pi = np.pi

In [2]:
signal_1 = np.array([0.2,      2*pi-0.1, 3.2, 2.9])
signal_2 = np.array([2*pi-0.2, 0.1,      2.8, 3.1])
signal   = np.column_stack((signal_1, signal_2))
print(signal)

[[0.2   6.083]
 [6.183 0.1  ]
 [3.2   2.8  ]
 [2.9   3.1  ]]


In [3]:
Theta = np.array(np.meshgrid(np.array([0, 1, 3]),
                             np.array([0, 1, 3]),
                             indexing='ij')).reshape(2, -1).T
print(Theta)

[[0 0]
 [0 1]
 [0 3]
 [1 0]
 [1 1]
 [1 3]
 [3 0]
 [3 1]
 [3 3]]


In [4]:
costs = np.empty((len(signal), len(Theta)))
for i, theta in enumerate(Theta):
    costs[:, i] = np.sum(np.square(np.minimum(np.abs(signal - theta), 2*pi - np.abs(signal - theta))), axis=1)
costs_df = pd.DataFrame(costs, index=[f'{signal[i]}' for i in range(len(signal))])
costs_df.columns = [f'{Theta[i]}' for i in range(len(Theta))]
costs_df

,[0 0],[0 1],[0 3],[1 0],[1 1],[1 3],[3 0],[3 1],[3 3]
[0.2 6.083],0.080,1.480,9.546,0.680,2.080,10.146,7.880,9.280,17.346
[6.183 0.1 ],0.020,0.820,8.420,1.220,2.020,9.620,9.620,10.420,18.020
[3.2 2.8],17.346,12.746,9.546,12.680,8.080,4.880,7.880,3.280,0.080
[2.9 3.1],18.020,12.820,8.420,13.220,8.020,3.620,9.620,4.420,0.020


In [5]:
cumsum_costs = get_cumsum(signal, Theta)
print(cumsum_costs)

[[ 0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.08   1.48   9.546  0.68   2.08  10.146  7.88   9.28  17.346]
 [ 0.1    2.3   17.966  1.9    4.1   19.766 17.5   19.7   35.366]
 [17.446 15.046 27.512 14.58  12.18  24.646 25.38  22.98  35.446]
 [35.466 27.866 35.932 27.8   20.2   28.266 35.    27.4   35.466]]


In [6]:
L([0], 2, cumsum_costs)

array([0.1])

In [7]:
L([1], 3, cumsum_costs)

array([10.1])

In [8]:
L([2], 4, cumsum_costs)

array([0.1])

In [9]:
lda = 1
cumsum_costs = get_cumsum(signal, Theta)  # get cumsum_costs matrix

# Get tau_star (list of best last changepoints)
C = np.zeros(len(signal) + 1)  # best cost for each segment from 0 to t
C[0] = -lda
tau_star = np.zeros(len(signal) + 1, dtype=int)  # initiate tau_star

for t in range(1, len(signal) + 1):
    indices = np.arange(t)
    L_values = L(indices, t, cumsum_costs)
    V = C[:t] + lda + L_values
    C[t] = np.min(V)
    tau_star[t] = np.argmin(V)

    print(f"Time step t = {t}")
    print(f"  Indices:     {indices}")
    print(f"  L values:    {L_values}")
    print(f"  V values:    {V}")
    print(f"  C[{t}]:        {C[t]:.2}")
    print(f"  tau_star[{t}]: {tau_star[t]}")
    print("-" * 40)

Time step t = 1
  Indices:     [0]
  L values:    [0.08]
  V values:    [0.08]
  C[1]:        0.08
  tau_star[1]: 0
----------------------------------------
Time step t = 2
  Indices:     [0 1]
  L values:    [0.1  0.02]
  V values:    [0.1 1.1]
  C[2]:        0.1
  tau_star[2]: 0
----------------------------------------
Time step t = 3
  Indices:     [0 1 2]
  L values:    [12.18 10.1   0.08]
  V values:    [12.18 11.18  1.18]
  C[3]:        1.2
  tau_star[3]: 2
----------------------------------------
Time step t = 4
  Indices:     [0 1 2 3]
  L values:    [20.2  18.12  0.1   0.02]
  V values:    [20.2 19.2  1.2  2.2]
  C[4]:        1.2
  tau_star[4]: 2
----------------------------------------
